# 2025-1 Taller de investigación IV (text-mining)

## Estudiante: Ferreyra, Lorenzo

En esta notebok se usa la librería [praw](https://praw.readthedocs.io/en/latest/) para extraer comentarios de un subreddit de Reddit mediante la API de Reddit. Anteriormente, se había propuesto una librería de R para extraer tweets de X, pero se encontraron dificultades técnicas a partir de un cambio que hizo Google en su navegador, específicamente en el modo headless empleado en Selenium.
El objetivo con el que se genera esta fuente de datos es analizar qué piensan los desarolladores y personal de IT sobre la inteligencia artificial generativa, los grandes modelos de lenguaje y sus productos derivados.

### Cómo accedo a la API de Reddit 

Para acceder a la API de Reddit necesitás un Client ID y un Client Secret. Para obtener estos datos, seguí las instrucciones en la sección **First Steps** [acá](https://github.com/reddit-archive/reddit/wiki/OAuth2-Quick-Start-Example). Antes de seguir los pasos, debés crear una cuenta en Reddit, iniciar sesión con ella y luego generar una app en la sección Developers.

Una vez que tengas las credenciales necesarias, lo más recomendable es guardarlas en un archivo .json para mantenerlas seguras. La estructura del archivo debe ser la siguiente:

```python

{"username":YourUsername, 
 "password": YourPassword, 
 "client_id":YourClientID, 
 "client_secret":YourClientSecret}
```

En adelante, se asume que esta información está guardada en un archivo llamado `reddit_credentials.json`

### Creación de una instancia de Reddit 

Para hacer uso de la API, debe crearse una instancia autorizada de `reddit` enviando las credenciales (almacenadas en `reddit_credentials.json`) hacia la clase Reddit de la librería praw. Como las credenciales están originalmente en un json, una vez se carga el archivo, se lo trata como un diccionario de Python.

```python
import praw
import json 

with open('reddit_credentials.json') as fin:
    creds = json.load(fin)
reddit = praw.Reddit(user_agent='Comment Extraction (by /u/{0})'.format(creds['username']),
                     client_id=creds['client_id'], client_secret=creds['client_secret'],
                     username=creds['username'], password=creds['password'])
```

### Acceder al Subreddit de nuestra preferencia 

Ahora se puede crear una instancia del `subreddit` que nos interese eligiendo su nombre real como argumento del método subreddit de nuestra instancia de `reddit`. Como me interesa conocer los comentarios y lo que piensan los desarrolladores y el personal IT en general, elijo el subreddit 'devsarg' que es uno de los más populares en Argentina, y de los más activos también en países limítrofes.

```python
subreddit = reddit.subreddit('devsarg')
```

### Buscar los comentarios que nos interesan dentro de ese subreddit

Este bloque de código lo que hace es una especie de "barrido" dentro del subreddit r/devsarg, que es una comunidad de desarrolladores de Argentina. La idea es recolectar qué se está diciendo sobre temas relacionados con inteligencia artificial generativa.

Para eso, se arma una búsqueda (una "query") con palabras clave como:
* gpt
* ia
* chatgpt
* ia generativa
* vibe coding
* cursor
* llms

Estas palabras están relacionadas con herramientas, tecnologías y conceptos que suelen aparecer en conversaciones sobre IA y programación.

El código va recorriendo los últimos 20 posts que coincidan con esas palabras y, por cada uno, guarda la información más importante (como el título, el link, la cantidad de votos, etc.) en un archivo llamado devsarg_search_results.json.

Ese archivo después se puede usar para analizar las publicaciones, ver qué temas aparecen más, cómo reaccionan los usuarios, o incluso entrenar algún modelo si quisieras.

```python
query = "gpt OR ia OR chatgpt OR ia generativa OR vibe coding OR cursor OR llms"

with open('devsarg_search_results.json', 'w') as fout:
    for submission in subreddit.search(query, sort="new", limit=20):
        thread = get_metadata(submission)
        json.dump(thread, fout)
        fout.write('\n')
```

La metadata que se puede extraer de un subreddit y sus comentarios es muy amplia, entonces achicamos el alcance a solo algunas de las opciones disponibles

```python
def get_metadata(submission):

    thread={}
    thread["title"]=submission.title
    thread["score"]=submission.score
    thread["id"]=submission.id
    thread["url"]=submission.url
    thread["comms_num"]=submission.num_comments
    thread["created"]=submission.created
    thread["body"]=submission.selftext

    return thread
```

La función va a devolver un diccionario que contiene metadatos sobre cada hilo (post).

Ahora podemos recorrer los 20 hilos más nuevos, obtener los metadatos de cada uno y guardar los resultados en un archivo llamado devsarg.json.

```python
with open('devsarg.json', 'w') as fout:
    for submission in subreddit.search(query, sort="new", limit=20):
        thread = get_metadata(submission)
        json.dump(thread, fout)
        fout.write('\n')
```

### Extraer comentarios de los posts 

Ahora podemos iterar sobre las `submissions` (publicaciones) dentro de un subreddit y obtener metadatos de cada una.

Para obtener los comentarios de cada `submission`, podemos usar el atributo `comments`. La estructura de los comentarios en un hilo de subreddit es similar a un árbol: un comentario principal puede tener muchas respuestas, y esas respuestas pueden tener otras respuestas. Por eso, necesitamos recorrer ese "árbol de comentarios" para obtenerlos todos.

Recorrer el árbol es fácil gracias al objeto `CommentForest` de `praw`. Para más detalles, podés ver este [ejemplo](https://praw.readthedocs.io/en/latest/tutorials/comments.html#extracting-comments) en la documentación oficial de `praw`.

Si mirás cómo está estructurado un hilo en devsarg (o en cualquier otro subreddit), vas a notar que para ver todos los comentarios tenés que hacer clic en '1 more comment', '21 more comments', etc.

Un punto importante a tener en cuenta es que, para obtener *todos* los comentarios de un hilo, necesitamos realizar una acción similar a esa. En `praw`, esto se puede lograr usando el método `replace_more`, que forma parte de `CommentForest`.

En concreto, para obtener todos los comentarios, podemos usar el atributo `comments` sobre cada `submission`, junto con el método `replace_more` y un límite de `None`. Esto solo se hace una vez, y luego ya se puede iterar sobre la lista de todos los comentarios y extraer los datos que necesitemos.

```python
def get_comments(submission):

    comments=[]

    submission.comments.replace_more(limit=None)
    for comment in submission.comments.list():
        data={}
        data['body']=comment.body
        data['id']=comment.fullname

        try:
            data['author']=comment.author.name

        except AttributeError:
            data['author']=comment.author

        data['time']=comment.created_utc
        data['parent']=comment.parent_id

        comments.append(data)

    return comments
```

### Implementacion

Puesta en práctica del código para generar la fuente de datos requerida

In [1]:
import praw
import json 

def get_comments(submission):

    comments=[]

    submission.comments.replace_more(limit=None)
    for comment in submission.comments.list():
        data={}
        data['body']=comment.body
        data['id']=comment.fullname

        try:
            data['author']=comment.author.name

        except AttributeError:
            data['author']=comment.author

        data['time']=comment.created_utc
        data['parent']=comment.parent_id

        comments.append(data)

    return comments


def get_metadata(submission):

    thread={}
    thread["title"]=submission.title
    thread["score"]=submission.score
    thread["id"]=submission.id
    thread["url"]=submission.url
    thread["comms_num"]=submission.num_comments
    thread["created"]=submission.created
    thread["body"]=submission.selftext
    
    thread['posts']= get_comments(submission)

    return thread



with open('reddit_credentials.json') as fin:
    
    creds = json.load(fin)
    

reddit = praw.Reddit(user_agent='Comment Extraction (by /u/{0})'.format(creds['username']),
                     client_id=creds['client_id'], client_secret=creds['client_secret'],
                     username=creds['username'], password=creds['password'])


subreddit = reddit.subreddit('devsarg')
query = "gpt OR ia OR chatgpt OR ia generativa OR vibe coding OR cursor OR llms"


with open('devsarg.json', 'w') as fout:
    for submission in subreddit.search(query, sort="new", limit=20):
        thread = get_metadata(submission)
        json.dump(thread, fout)
        fout.write('\n')